In [ ]:
import hierarchical_genomes as hg
import numpy as np
import copy
import reservoirpy as rpy
from reservoirpy.nodes import Ridge, Reservoir
from reservoirpy.datasets import mackey_glass
from reservoirpy.observables import rmse
import matplotlib.pyplot as plt

genome = [[0,1,0.5],[1,2,0.5],[2,3,0.1],[0,1,0.5],[1,2,0.5],[2,3,0.1],[0,1,0.5],[1,2,0.5],[2,3,0.1]]
n_generations = 100
population_size = 50 
genome_population = [copy.deepcopy(genome) for i in range(population_size)]
X = mackey_glass(2000)  # Double the data length

# Normalize data between 0 and 1
X = (X - np.min(X)) / (np.max(X) - np.min(X))

fitness_history = []

for generation in range(n_generations):
    list_of_genomes = []
    list_of_fitness = []
    
    for genome in genome_population:
        mutated_genome, mutation = hg.mutate_genome_with_hox(genome)
        weight_matrix = hg.transcribe_hierarchical_genome_to_weight_matrix(mutated_genome)
        
        # Adjust hyperparameters for reservoir and ridge regression
        reservoir = Reservoir(W=weight_matrix, sr=0.9, leak_rate=0.7)  
        readout = Ridge(ridge=1e-5)
        esn = reservoir >> readout
        
        # Correct the temporal misalignment
        forecast = esn.fit(X[:900], X[1:901]).run(X[900:1900])
        fitness = rmse(forecast, X[901:1901])
        
        list_of_genomes.append(mutated_genome)
        list_of_fitness.append(fitness)

    best_rmse_index = np.argmin(list_of_fitness)
    best_genome = list_of_genomes[best_rmse_index]
    best_fitness = list_of_fitness[best_rmse_index]
    fitness_history.append(best_fitness)
    
    # Create a new generation with a mix of best genome and some mutations
    genome_population = [copy.deepcopy(best_genome) if np.random.rand() < 0.8 else hg.mutate_genome_with_hox(best_genome)[0] for i in range(population_size)]
    
plt.plot(fitness_history)
plt.xlabel("Generations")
plt.ylabel("Best RMSE")
plt.title("Fitness History over Generations")
plt.show()
